# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
ls

 驱动器 C 中的卷是 OS
 卷的序列号是 A0F4-25BA

 C:\Users\BobKing\Documents\GitHub\guoqingbao.github.io\markdown_generator 的目录

2021/03/09  21:44    <DIR>          .
2021/03/09  21:44    <DIR>          ..
2021/03/09  21:44    <DIR>          .ipynb_checkpoints
2019/06/17  22:33            12,858 publications.ipynb
2021/03/09  21:43             3,889 publications.py
2021/03/09  21:41             1,905 publications.tsv
2019/06/17  22:33             8,764 PubsFromBib.ipynb
2019/06/17  22:33             6,018 pubsFromBib.py
2019/06/17  22:33               492 readme.md
2019/06/17  22:33            13,836 talks.ipynb
2019/06/17  22:33             4,000 talks.py
2019/06/17  22:33             1,188 talks.tsv
               9 个文件         52,950 字节
               3 个目录 97,173,012,480 可用字节


## Import pandas

We are using the very handy pandas library for dataframes.

In [14]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [21]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2021-03-01,COVID-MTL: Multitask Learning with Shift3D and...,Patterm Recognition,This paper is about COVID-10 diagnosis with de...,"Guoqing Bao, Huai Chen, Tongliang Liu, Guanzho...",COVIDMTL,https://arxiv.org/abs/2012.05509
1,2021-02-09,PathoFusion: Recognition of pathomorphological...,Cancers,We proposed an AI framework for cross-modality...,"Guoqing Bao, Xiuying Wang, Ran Xu, Christina L...",PathoFusion,https://doi.org/10.3390/cancers13040617
2,2020-10-01,Identification of lncRNA Signature Associated ...,IEEE Journal of Biomedical and Health Informatics,We proposed a machine learning based framework...,"Guoqing Bao, Ran Xu, Xiuying Wang, Jianxiong J...",Pancancer,https://doi.org/10.1109/JBHI.2020.3027680
3,2020-11-1,Depthwise Multiception Convolution for Reducin...,"The 16th International Conference on Control, ...",We proposed a new convolutional method to impr...,"Guoqing Bao, Manuel B. Graeber, Xiuying Wang (...",Multiception,https://doi.org/10.1109/ICARCV50220.2020.9305369
4,2020-11-2,A Bifocal Classification and Fusion Network fo...,"The 16th International Conference on Control, ...",We proposed a bifocal classification and fusio...,"Guoqing Bao, Manuel B. Graeber, Xiuying Wang (...",Bifocal,https://doi.org/10.1109/ICARCV50220.2020.9305360


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [22]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [23]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [18]:
ls "../_publications/"

 驱动器 C 中的卷是 OS
 卷的序列号是 A0F4-25BA

 C:\Users\BobKing\Documents\GitHub\guoqingbao.github.io\_publications 的目录

2021/03/09  21:49    <DIR>          .
2021/03/09  21:49    <DIR>          ..
2019/06/17  22:33               630 2009-10-01-paper-title-number-1.md
2019/06/17  22:33               630 2010-10-01-paper-title-number-2.md
2019/06/17  22:33               630 2015-10-01-paper-title-number-3.md
2021/03/09  21:49             1,188 2020-10-01-Pancancer.md
2021/03/09  21:45             1,199 2020-10-01-paper-title-number-3.md
2021/03/09  21:49             1,389 2021-02-09-PathoFusion.md
2021/03/09  21:45             1,387 2021-02-09-Published.md
2021/03/09  21:49             1,153 2021-03-01-COVIDMTL.md
2021/03/09  21:45             1,159 2021-30-01-Major Revision.md
               9 个文件          9,365 字节
               2 个目录 97,218,191,360 可用字节


In [13]:
cat "../_publications/2021-02-09-Published.md"

SyntaxError: invalid syntax (<ipython-input-13-066e84916de3>, line 1)